In [1]:
%matplotlib inline
from IPython.display import display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

con = sqlite3.connect("../data/CongoDB.sqlite")

Kartierung Quellenstand
* welche Fpl kennen wir
* wo wurde gegraben
* wo wurde Geomagnetik gemacht (bislang nur in IYO)

In [2]:
sql = """SELECT
           t_Ort.ort_name AS SITE,
           t_Ort.x_long AS X,
           t_Ort.y_lat AS Y
       FROM t_Ort
       WHERE ((t_Ort.x_long) != '')""".replace('\n',' ')

df1 = pd.read_sql(sql, con)
df1['TYPE'] = 'all'
# df1.to_csv('../data/processed/Fpl_Quellenlage_All.csv', index=False)
display(df1.head())

,SITE,X,Y,TYPE
0,Gombe,17.580406,-0.704069,all
1,Batalimo,18.455022,3.675942,all
2,Mondjo,19.269369,0.347772,all
3,Bojenjo,17.432057,-0.006791,all
4,Bokuma,17.478148,0.099491,all


In [3]:
len(df1.index)

756

In [4]:
sql = """SELECT
           t_Ort.ort_name AS SITE,
           t_Ort.x_long AS X,
           t_Ort.y_lat AS Y,
           t_Komplex.bef_art
       FROM t_Ort INNER JOIN t_Komplex ON t_Ort.ortID = t_Komplex.ortID
       WHERE (((t_Komplex.bef_art) Not Like '%Oberfl%')
           AND ((t_Komplex.bef_art) Not Like '%Obfl%')
           AND ((t_Komplex.bef_art) Not Like '%rezent%')
           AND ((t_Komplex.bef_art) Not Like '%ethno%')
           AND ((t_Komplex.bef_art) Not Like '%Geoprofil%')
           AND ((t_Komplex.bef_art) Not Like '%Pollen%'))""".replace('\n',' ')

df2 = pd.read_sql(sql, con)
df2 = df2.reset_index()
display(df2.head())

,index,SITE,X,Y,bef_art
0,0,Bamanya,18.316297,0.009790,Ofen
1,1,Bobusa,16.917098,-1.052349,Schicht
2,2,Bobusa,16.917098,-1.052349,Schicht
3,3,Boleko,17.221926,-0.805556,Grab
4,4,Likwala-Esobe km 186,17.406785,-0.048312,Grube


In [5]:
df2.pivot_table(values = 'index', index = 'bef_art', aggfunc = len, margins = all)

,index
bef_art,
Abris,1.0
Camp Site,1.0
Ceramic Feature,1.0
Depot,2.0
Erzpinge,1.0
Geologic Profile,1.0
Grab,37.0
Graben,2.0
Grube,175.0


In [6]:
df2_pivot = df2.pivot_table(values = 'index', index = ['SITE', 'X', 'Y'], aggfunc = len)
df2_pivot = df2_pivot.reset_index()
df2_pivot = df2_pivot[['SITE', 'X', 'Y']]
df2_pivot['TYPE'] = 'excavation'
# df2_pivot.to_csv('../data/processed/Fpl_Quellenlage_Excavations.csv', index=False)
display(df2_pivot.head())

,SITE,X,Y,TYPE
0,Abang Minko'o,11.422917,2.329389,excavation
1,Akonetye,11.174669,2.475500,excavation
2,Bamanya,18.316297,0.009790,excavation
3,Bampoko,18.801017,-0.146343,excavation
4,Baombi II,25.160636,0.656695,excavation


In [7]:
# Magnetik bislang nur in IYO
sql = """SELECT
           t_Ort.ort_name AS SITE,
           t_Ort.x_long AS X,
           t_Ort.y_lat AS Y
       FROM t_Ort
       WHERE ((t_Ort.ort_name) = 'Iyonda')""".replace('\n',' ')

df3 = pd.read_sql(sql, con)
df3['TYPE'] = 'magnetic survey'
# df3.to_csv('../data/processed/Fpl_Quellenlage_magenticSurvey.csv', index=False)
df3.head()

,SITE,X,Y,TYPE
0,Iyonda,18.184492,-0.037076,magnetic survey


In [8]:
df = pd.concat([df1, df2_pivot, df3])
df.to_csv('../data/processed/Fpl_Quellenlage.csv', index=False)
display(df.head())

,SITE,X,Y,TYPE
0,Gombe,17.580406,-0.704069,all
1,Batalimo,18.455022,3.675942,all
2,Mondjo,19.269369,0.347772,all
3,Bojenjo,17.432057,-0.006791,all
4,Bokuma,17.478148,0.099491,all


## Nur im Arbeitsgebiet

In [9]:
sql = """SELECT
           t_Ort.ort_name AS SITE,
           t_Ort.x_long AS X,
           t_Ort.y_lat AS Y
       FROM t_Ort
       WHERE (((t_Ort.x_long) != '')
           AND ((t_ort.ort_lit) = 'DS'))""".replace('\n',' ')

df1 = pd.read_sql(sql, con)
df1['TYPE'] = 'all'
len(df1.index)

125

In [10]:
sql = """SELECT
           t_Ort.ort_name AS SITE,
           t_Ort.x_long AS X,
           t_Ort.y_lat AS Y,
           t_Komplex.bef_art
       FROM t_Ort INNER JOIN t_Komplex ON t_Ort.ortID = t_Komplex.ortID
       WHERE ((((t_ort.ort_lit) = 'DS')
           AND (t_Komplex.bef_art) Not Like '%Oberfl%')
           AND ((t_Komplex.bef_art) Not Like '%Obfl%')
           AND ((t_Komplex.bef_art) Not Like '%rezent%')
           AND ((t_Komplex.bef_art) Not Like '%ethno%')
           AND ((t_Komplex.bef_art) Not Like '%Geoprofil%')
           AND ((t_Komplex.bef_art) Not Like '%Pollen%'))""".replace('\n',' ')

df2 = pd.read_sql(sql, con)
df2 = df2.reset_index()
len(df2.index)

19

In [11]:
df2.pivot_table(values = 'index', index = 'bef_art', aggfunc = len, margins = all)

,index
bef_art,
Grab,2.0
Grube,9.0
Grube (?),2.0
Grube/Ofen,2.0
Ofen,2.0
Schicht,2.0
All,19.0


In [12]:
df2_pivot = df2.pivot_table(values = 'index', index = ['SITE', 'X', 'Y'], aggfunc = len)
df2_pivot = df2_pivot.reset_index()
df2_pivot = df2_pivot[['SITE', 'X', 'Y']]
df2_pivot['TYPE'] = 'excavation'

In [13]:
df = pd.concat([df1, df2_pivot])
df.to_csv('../data/processed/Fpl_Quellenlage_ArbeitsgebietDS.csv', index=False)
display(df.head())

,SITE,X,Y,TYPE
0,Gombe,17.580406,-0.704069,all
1,Bojenjo,17.432057,-0.006791,all
2,Bokuma,17.478148,0.099491,all
3,Boleko,17.221926,-0.805556,all
4,Botwale,17.365479,-0.552494,all


# Wotzka 1995

In [17]:
sql = """SELECT
           t_Ort.ort_name AS SITE,
           t_Ort.x_long AS X,
           t_Ort.y_lat AS Y,
           t_Komplex.bef_art
       FROM t_Ort INNER JOIN t_Komplex ON t_Ort.ortID = t_Komplex.ortID
       WHERE ((((t_ort.ort_lit) LIKE '%Wotzka%')
           AND (t_Komplex.bef_art) Not Like '%Oberfl%')
           AND ((t_Komplex.bef_art) Not Like '%Obfl%')
           AND ((t_Komplex.bef_art) Not Like '%rezent%')
           AND ((t_Komplex.bef_art) Not Like '%ethno%')
           AND ((t_Komplex.bef_art) Not Like '%Geoprofil%')
           AND ((t_Komplex.bef_art) Not Like '%Pollen%'))""".replace('\n',' ')

df = pd.read_sql(sql, con)
df = df.reset_index()
df.head()

,index,SITE,X,Y,bef_art
0,0,Bamanya,18.316297,0.009790,Ofen
1,1,Mondjo,19.269369,0.347772,Graben
2,2,Mondjo,19.269369,0.347772,Grube
3,3,Mondjo,19.269369,0.347772,Grube
4,4,Bamanya,18.316297,0.009790,Ofen


In [19]:
len(df.pivot_table(values = 'index', index = 'SITE', aggfunc = len))

20